## Simple Gen AI Web Scraper using Langchain

In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
os.environ['OPENAI_API_KEY']        = os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY']     = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_PROJECT']     = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]  = "true"

## Data Ingestion -- Scrape Website Data

In [ ]:
from langchain_community.document_loaders           import WebBaseLoader
from langchain_text_splitters                       import RecursiveCharacterTextSplitter
from langchain_openai                               import OpenAIEmbeddings
from langchain_community.vectorstores               import FAISS
from langchain.chains.combine_documents             import create_stuff_documents_chain
from langchain_core.prompts                         import ChatPromptTemplate
from langchain_openai                               import ChatOpenAI
from langchain_core.documents                       import Document
from langchain.chains                               import create_retrieval_chain

In [ ]:
url = "https://docs.smith.langchain.com/administration/tutorials/manage_spend"

loader = WebBaseLoader(url)
loader

In [ ]:
loadedDocument = loader.load()
loadedDocument

In [ ]:
textSplitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

In [ ]:
documents = textSplitter.split_documents(loadedDocument)
documents

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
vectorStoreDB = FAISS.from_documents(
    documents, 
    embeddings
)

## Query and Retrieval

In [ ]:
llmModel = ChatOpenAI(model = "gpt-4o")

In [ ]:
query   = "LangSmith has two usage limits: total traces and extended"

result  = vectorStoreDB.similarity_search(query)
print(f'Result:     {result[0].page_content}')

In [ ]:
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context: 
    <context>
    {context}
    </context>
    """
)

In [ ]:
documentChain = create_stuff_documents_chain(
    llmModel,
    prompt
)

documentChain

In [ ]:
documentChain.invoke({
    "input": "Langsmith has two usage limits: total traces and extended",
    "context": [Document(
        page_content = "These correspond to the two metrics we've been tracking on our usage graph. We can use these in tandem to have granular control over spend.")]
})

In [ ]:
retriever = vectorStoreDB.as_retriever()

retreivalChain = create_retrieval_chain(
    retriever,
    documentChain   
)
retreivalChain

## Response

In [ ]:
response = retreivalChain.invoke({
    "input": "Langsmith has two usage limits: total traces and extended"
})

response

In [ ]:
response['answer']

In [ ]:
response['context']